# 第四步：后处理



在这一部分中，我们对优化结果进行后处理以进行评估和可视化。
首先，我们准备以下参数：

|参数 |说明 |数值 |
|--- |--- |--- |
|method | QUBO问题的退火方法|'dwave-sa'：使用Ocean工具包中的模拟退火器<br>'dwave-qa'：使用量子退火器|
|raw_path|原始分子文件的路径| 在这个例子中是'./molecule-data/117_ideal.mol2'|
|data_path|处理后的分子文件的路径| 在这个例子中是'./qmu_117_ideal_data_latest.mol2'|
|bucket |存储结果的 s3 存储桶 | - |
|prefix | s3 存储桶中的文件夹名称 | - |
|task_id |您的量子退火任务id| 在这个例子中是'2b5a3b05-1a0e-443a-852c-4ec422a10e59'|

然后我们可以使用 SA 的 **ResultParser** 对象运行后处理：

In [ ]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [18]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
# print(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:_load_raw_result
INFO:root:load simulated annealer raw result
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse simulated annealer result
INFO:root:sa only has local_time!


time for dwave-sa: 
     local time is 175.73437476158142


In [19]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_process_result.save_mol_file(f"{timestamp}")


INFO:root:generate_optimize_pts()
INFO:numexpr.utils:NumExpr defaulting to 2 threads.
INFO:root:save_mol_file 20220216-06
INFO:root:finish save ./molecule-data/117_ideal_dwave-sa_20220216-06.mol2 and ./molecule-data/117_ideal_dwave-sa_20220216-06.json


['./molecule-data/117_ideal_dwave-sa_20220216-06.mol2',
 './molecule-data/117_ideal_dwave-sa_20220216-06.json']

In [20]:
sa_process_result.parameters

{'volume': {'optimize': 18.9423642473021,
  'initial': 18.549876807564765,
  'gain': 1.0211584930621898,
  'unfolding_results': ['x_15_7', 'x_14_8']}}

在第一个代码块中，我们可以看到SA的**local time**
大约是 174 秒。
使用 **generate_optimize_pts()** 方法，最终的 3D
展开后的点会生成并保存为json文件和mol2文件。
最后一个代码块显示优化结果，也存储在 json 文件中。从结果看出体积增加
1.0212倍。 **unfolding_results** 的值表示
可旋转键 15 应旋转 $270^o$ ($360/8*(7-1)$) 和
可旋转键 14 应旋转 $315^o$ ($360/8*(8-1)$)。
同时，您可以运行 QA 的后处理：

In [21]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
# print(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

print(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

INFO:root:_load_raw_result
INFO:root:load quantum annealer raw result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/89a59fa9-c66e-482d-85e0-6dfce8c54454/qa_result.pickle
INFO:root:MoleculeData.load()
INFO:root:init mol data for final position
INFO:root:init mol data for raw position
INFO:root:_parse_model_info
INFO:root:_init_parameters
INFO:root:parse quantum annealer result
INFO:root:_read_result_obj
INFO:root:_read_result_obj: annealer-experiment/89a59fa9-c66e-482d-85e0-6dfce8c54454/results.json


time for dwave-qa: 
     local time is 7.57835578918457,
     task time is 1.89, 
     qpu total time is 0.142533, 
     qpu access time is 0.124129


我们可以看到运行 QA 的时间指标有很多种。
该任务的**local time**为 7.7 秒，即调用 api 和
得到退火结果的时间。 **task time**  是来自存放在桶里面的json 文件的度量。我们还可以看到**qpu total time**和**qpu access time**代表
在 QPU 中运行的实际时间。请参考[操作与时序](https://docs.dwavesys.com/docs/latest/c_qpu_timing.html)的详情。

In [22]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_process_result.save_mol_file(f"{timestamp}")

INFO:root:generate_optimize_pts()
INFO:root:save_mol_file 20220216-06
INFO:root:finish save ./molecule-data/117_ideal_dwave-qa_20220216-06.mol2 and ./molecule-data/117_ideal_dwave-qa_20220216-06.json


['./molecule-data/117_ideal_dwave-qa_20220216-06.mol2',
 './molecule-data/117_ideal_dwave-qa_20220216-06.json']

In [23]:
qa_process_result.parameters

{'volume': {'optimize': 18.9423642473021,
  'initial': 18.549876807564765,
  'gain': 1.0211584930621898,
  'unfolding_results': ['x_15_7', 'x_14_8']}}

同理，优化后的结果被转化为3D点并保存
作为本地json和mol2文件。结果表明QA获得了
1.021倍
体积增加。



最后，我们可以打开文件夹以获得优化的结果：

![optimize-results](../../../images/optimize-results.png)

 <center>优化结果</center>

我们可以将 SA 和 QA 的 json 结果和 mol2 文件分别存放在这个地方。
如果我们执行更多实验，更多带有时间戳的结果会存储在这里。
我们可以上传
结果**117_ideal_dwave-qa_20220216-05.mol2**
进入
[在线查看工具](https://www.rcsb.org/3d-view)
查看结果：

![视觉](../../../images/visualization.png)

 <center>可视化</center>